**<font size="5">Applied Statistics</font>**

<font size="3">MSc in High Performance Computing Engineering, Computer Science and Engineering, Physics Engineering - A.Y. 2024-2025</font>

Prof. Mario Beraha - Dott. Vittorio Torri

---

<font size="4">**Lab 6 - Linear Regression**</font>

# Libraries

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
import statsmodels.api as sm

In [ ]:
np.random.seed(1234)

In [ ]:
import scipy.stats as stats

# Load Dataset

In [ ]:
df = pd.read_csv('../DatasetsLabs/heart_failure_clinical_records_dataset_smhd.csv')
df

In [ ]:
cat_vars = ['anaemia', 'diabetes', 'high_blood_pressure',  'sex',  'smoking',  'DEATH_EVENT']
num_vars = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'bmi', 'time']

# Simple Linear Regression

Let's look at the scatterplot of bmi and ejection fraction

In [ ]:
TARGET = 'bmi'

In [ ]:
sns.relplot(x="ejection_fraction", y="bmi", palette="muted", height=6, data=df)

It looks like there is a relationship between them: we can try to predict BMI based on ejection fraction

In [ ]:
X = df['ejection_fraction'].values.reshape(-1,1) #da un array 1d ad uno 2d con una colonna
y = df['bmi']
y.shape, X.shape

In [ ]:
X_const = sm.add_constant(X)

In [ ]:
X_const.shape, y.shape, type(X_const)

$$ bmi = \beta_0 + \beta_1 \cdot ef + \epsilon $$

In [ ]:
model = sm.OLS(y, X_const)
results = model.fit()
print(results.summary())


In [ ]:
y_pred = results.predict(X_const)

r2 = r2_score(y,y_pred)
mse = mean_squared_error(y,y_pred)

print(f'R2: {r2:.4f}')
print(f'MSE: {mse:.4f}')

In [ ]:
X_plot = np.arange(np.min(X),np.max(X),0.1).reshape(-1,1)
X_plot = sm.add_constant(X_plot)
y_plot = results.predict(X_plot)

plt.scatter(X, y)
plt.plot(X_plot[:,1], y_plot, c='blue')

In [ ]:
ef_point = 45
X0_const = np.array([1, ef_point])
prediction = results.get_prediction(X0_const) #ritorna una oggetto di una classe
intervals = prediction.summary_frame(alpha=0.05)
intervals
prediction = intervals['mean']
ci_lower = intervals['mean_ci_lower']
ci_upper = intervals['mean_ci_upper']
pi_lower = intervals['obs_ci_lower']
pi_upper = intervals['obs_ci_upper']

#plotting
plt.scatter(X0_const[1], prediction, c='red')
plt.scatter(X0_const[1], ci_lower, c='blue')
plt.scatter(X0_const[1], ci_upper, c='blue')
plt.scatter(X0_const[1], pi_lower, c='green')
plt.scatter(X0_const[1], pi_upper, c='green')
plt.show()

## Confidence/Prediction intervals

In [ ]:
X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
X_plot_const = sm.add_constant(X_plot)

# Predict with confidence intervals
predictions = results.get_prediction(X_plot_const)
summary_frame = predictions.summary_frame(alpha=0.05)  # 95% intervals
summary_frame


In [ ]:
# Extract the values
y_pred = summary_frame['mean']
ci_lower = summary_frame['mean_ci_lower']
ci_upper = summary_frame['mean_ci_upper']
pi_lower = summary_frame['obs_ci_lower']
pi_upper = summary_frame['obs_ci_upper']

plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['ejection_fraction'], y=df['bmi'], label='Data', color='black')
plt.plot(X_plot, y_pred, color='red', label='Regression Line')
plt.fill_between(X_plot.flatten(), ci_lower, ci_upper, color='blue', alpha=0.3, label='95% Confidence Interval')
plt.fill_between(X_plot.flatten(), pi_lower, pi_upper, color='orange', alpha=0.2, label='95% Prediction Interval')
plt.xlabel('Ejection Fraction')
plt.ylabel('BMI')
plt.title('Linear Regression with Confidence and Prediction Intervals')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
x_mean = X.mean()
idx_closest = np.argmin(np.abs(X_plot.flatten() - x_mean))

ci_width_at_mean = ci_upper.iloc[idx_closest] - ci_lower.iloc[idx_closest]
pi_width_at_mean = pi_upper.iloc[idx_closest] - pi_lower.iloc[idx_closest]

print(f'CI width at mean(X) = {x_mean:.2f}: {ci_width_at_mean:.4f}')
print(f'PI width at mean(X) = {x_mean:.2f}: {pi_width_at_mean:.4f}')

print("CI width (start/end):", ci_upper.iloc[0] - ci_lower.iloc[0], ci_upper.iloc[-1] - ci_lower.iloc[-1])
print("PI width (start/end):", pi_upper.iloc[0] - pi_lower.iloc[0], pi_upper.iloc[-1] - pi_lower.iloc[-1])

Prediction and confidence intervals for a pecific point, ef=45

In [ ]:
ef_point = 45
X_new = [1, ef_point] # Add constant manually

In [ ]:
### TODO
ef_point = 45
X_new = np.array([[1, ef_point]])  # Add constant manually

# Get predictions with confidence and prediction intervals
pred_result = results.get_prediction(X_new)
pred_summary = pred_result.summary_frame(alpha=0.05)

# Extract predicted values and intervals
mean_pred = pred_summary['mean'].values[0]
ci_lower = pred_summary['mean_ci_lower'].values[0]
ci_upper = pred_summary['mean_ci_upper'].values[0]
pi_lower = pred_summary['obs_ci_lower'].values[0]
pi_upper = pred_summary['obs_ci_upper'].values[0]

print(f"EF = {ef_point}")
print(f"Predicted BMI: {mean_pred:.2f}")
print(f"95% Confidence Interval (mean): [{ci_lower:.2f}, {ci_upper:.2f}]")
print(f"95% Prediction Interval (new patient): [{pi_lower:.2f}, {pi_upper:.2f}]")

# --- Plot ---
plt.figure(figsize=(8,6))
sns.scatterplot(x=df['ejection_fraction'], y=df['bmi'], label='Data', color='gray')
plt.axvline(x=ef_point, color='grey', linestyle='--', label=f'EF = {ef_point}')


# PI for new observation
plt.errorbar(ef_point, mean_pred,
             yerr=[[mean_pred - pi_lower], [pi_upper - mean_pred]],
             fmt='o', color='orange', capsize=10, alpha=0.5, label='95% PI (New Obs)')

# CI for mean
plt.errorbar(ef_point, mean_pred,
             yerr=[[mean_pred - ci_lower], [ci_upper - mean_pred]],
             fmt='o', color='red', capsize=5, label='95% CI (Mean)')

plt.xlabel('Ejection Fraction')
plt.ylabel('BMI')
plt.title(f'Prediction at EF = {ef_point}: Mean & New Observation Intervals')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Train-test set

So far we are evaluating the model on the same data it was trained on..

In [ ]:
from sklearn import model_selection

X = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'ejection_fraction', 'time']]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state=1234)

train_index = X_train.index
test_index = X_test.index

In [ ]:
### TODO: FIT THE MODEL ON TRAINING DATA
X_train_1 = X_train[['ejection_fraction']]
X_test_1 = X_test[['ejection_fraction']]
X_train_1 = sm.add_constant(X_train_1)
X_test_1 = sm.add_constant(X_test_1)

model = sm.OLS(y_train, X_train_1)
results = model.fit()
print(results.summary())

In [ ]:
### TODO: COMPUTE MSE AND R^2 ON TEST DATA
y_pred = results.predict(X_test_1)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
print(r2)
print(mse)

# Multiple linear regression

## Multiple input variables

In [ ]:
variables = num_vars.copy()
variables.remove('bmi')

X_train_2 = X_train[variables]
X_test_2 = X_test[variables]
X_train_2 = sm.add_constant(X_train_2)
X_test_2 = sm.add_constant(X_test_2)

In [ ]:
model = sm.OLS(y_train, X_train_2)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_2)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

n = len(y_test)
p = X_test_2.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

$$ R^2 = 1 - \frac{RSS}{TSS} = 1 - \frac{\sum{(y_i - \hat{y_i})^2}}{\sum{(y_i - \bar{y})^2}}$$

$$ AdjR^2 = 1 - \frac{n - 1}{n - k -1} \frac{RSS}{TSS} = 1 - \frac{n - 1}{n - k -1} (1 - R^2)$$

where $k$ is the number of regressors

## Collinearity

The Variance Inflation Factor (VIF) indicates how good a variable can be predicted from the others

A high VIF means there is collinearity with some other variable

Rule of thumb: VIF > 5 for a variable is problematic

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

data = X_train_2

vif_data = pd.DataFrame()
vif_data["Variable"] = data.columns

vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]

print(vif_data)

The correlation coefficients can also identify collinearity, but not always. If a variable X1 is a linear transformation of a variable X2, their linear correlation coefficient will be high. But if the collinearity involves a group of variable, then it might not be evident from the correlation matrix

In [ ]:
sns.heatmap(df[num_vars].corr(), cmap="Blues",annot=True)

## Scale variables

Numerical variables having different scales are often problematic. Many models have their coefficients' computation affected by this, but while this is not the case of linear regression with least square method, it is still useful to **standardize** the features for interpretability of coefficients and to reduce the condition numer when it is not due to a direct multicollinearity among the variables. The VIF is scale invariant, but the condition number is not.

Moreover, this is necessary when penalizations are introduced in linear regression models (more on this in the next lectures)

To standardize a variable x:

$$ z = \frac{x - \mu }{\sigma}$$

$$ $$

In [ ]:
X_train_2 = X_train[variables]
X_test_2 = X_test[variables]

scaler = StandardScaler()
X_train_2_scaled = pd.DataFrame(scaler.fit_transform(X_train_2), columns=X_train_2.columns, index=X_train_2.index)
X_test_2_scaled = pd.DataFrame(scaler.transform(X_test_2), columns=X_test_2.columns, index=X_test_2.index)

X_train_2_scaled = sm.add_constant(X_train_2_scaled)
X_test_2_scaled = sm.add_constant(X_test_2_scaled)


In [ ]:
model = sm.OLS(y_train, X_train_2_scaled)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_2_scaled)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_2_scaled.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

## Reduce the number of variables

### EF + creatinine

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \epsilon $$

In [ ]:
variables = ['ejection_fraction', 'serum_creatinine']

X_train_3 = X_train[variables]
X_test_3 = X_test[variables]
X_train_3 = sm.add_constant(X_train_3)
X_test_3 = sm.add_constant(X_test_3)

In [ ]:
model = sm.OLS(y_train, X_train_3)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_3)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_3.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

with 2 regressors we can still visualize a plane:

In [ ]:
# TODO
x1 = X_train_3['ejection_fraction']
x2 = X_train_3['serum_creatinine']
y = y_train

# Get model coefficients
b0, b1, b2 = results.params

# Create a grid for plotting the regression plane
x1_grid, x2_grid = np.meshgrid(
    np.linspace(x1.min(), x1.max(), 30),
    np.linspace(x2.min(), x2.max(), 30)
)
x2_grid.shape, y.shape,

In [ ]:
# Compute predicted y values over the grid
y_pred_grid = b0 + b1 * x1_grid + b2 * x2_grid
y_pred_grid.shape, b1, b0


In [ ]:
# Plot
fig = plt.figure(figsize=(16, 12))

views = [
    (0, 45),
    (20, 135),
    (0, 0),
    (10, 220)
]

titles = [
    "Regression Plane View 1",
    "Regression Plane View 2",
    "Regression Plane View 3",
    "Regression Plane View 4"
]

for i, (elev, azim) in enumerate(views):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    ax.scatter(x1, x2, y, color='r', label='Data')
    ax.plot_surface(x1_grid, x2_grid, y_pred_grid, alpha=0.4)
    ax.set_xlabel('Ejection Fraction')
    ax.set_ylabel('Serum Creatinine')
    ax.set_zlabel('Target')
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(titles[i])

plt.tight_layout()
plt.show()

with standardization:

In [ ]:
scaletor = StandardScaler()
X_train_scaled = scaletor.fit_transform(X_train[num_vars])
X_train_scaled #ci sta che i valori non siano compresi tra -1 e 1, hanno solo media 0 e deviazione standard 1

In [ ]:
X_train_3 = X_train[variables]
X_test_3 = X_test[variables]

scaler = StandardScaler()
X_train_3_scaled = pd.DataFrame(scaler.fit_transform(X_train_3), columns=X_train_3.columns, index=X_train_3.index)
X_test_3_scaled = pd.DataFrame(scaler.transform(X_test_3), columns=X_test_3.columns, index=X_test_3.index)

X_train_3_scaled = sm.add_constant(X_train_3_scaled)
X_test_3_scaled = sm.add_constant(X_test_3_scaled)



In [ ]:
model = sm.OLS(y_train, X_train_3_scaled)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_3_scaled)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_3_scaled.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

### EF + creatinine + time

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \beta_3 \cdot time + \epsilon $$

In [ ]:
# TODO

## Polynomial features

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot ef^2 + \epsilon $$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
variables = ['ejection_fraction']

X_train_6 = X_train[variables]
X_test_6 = X_test[variables]

polynomial2 = PolynomialFeatures(degree=2, include_bias=True)
X_train_6 = polynomial2.fit_transform(X_train_6)
X_test_6 = polynomial2.fit_transform(X_test_6)

In [ ]:
variables = ['ejection_fraction']

X_train_6 = X_train[variables]
X_test_6 = X_test[variables]

polynomial2 = PolynomialFeatures(degree=2, include_bias=True)
X_train_6 = polynomial2.fit_transform(X_train_6)
X_test_6 = polynomial2.fit_transform(X_test_6)

scaler = StandardScaler()

# Scale everything except the constant term (column index 0)
X_train_6[:, 1:] = scaler.fit_transform(X_train_6[:, 1:])
X_test_6[:, 1:] = scaler.transform(X_test_6[:, 1:])

# Convert to DataFrame, maintaining the column names
cols = ['const', 'ef', 'ef^2']
#cols = polynomial2.get_feature_names_out()
X_train_6 = pd.DataFrame(X_train_6, columns=cols, index=train_index)
X_test_6 = pd.DataFrame(X_test_6, columns=cols, index=test_index)

In [ ]:
X_train_6

In [ ]:
model = sm.OLS(y_train, X_train_6)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_6)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_6.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

In [ ]:
## TODO: plot the line fitted by this polinomial regression, compare to the simple regression on ef
X_plot = np.arange(np.min(X_train['ejection_fraction']),np.max(X_train['ejection_fraction']),0.1).reshape(-1,1)
X_plot_6 = polynomial2.transform(X_plot)

X_plot_6_scaled = X_plot_6
X_plot_6_scaled[:,1:] = scaler.transform(X_plot_6_scaled[:,1:])
y_plot_6 = results.predict(X_plot_6_scaled)

sns.relplot(y="bmi", x="ejection_fraction", palette="muted",
            height=6, data=df)
sns.lineplot(x=X_plot[:,0], y=y_plot_6,color='red')
sns.lineplot(x=X_plot[:,0], y=y_plot,color='green')

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \beta_3 \cdot ef^2 + \beta_4 \cdot ef \cdot creatinine + \beta_5 \cdot creatinine^2 + \epsilon $$

In [ ]:
variables = ['ejection_fraction', 'serum_creatinine']

X_train_7 = X_train[variables]
X_test_7 = X_test[variables]

polynomial2 = PolynomialFeatures(degree=2, include_bias=True)
X_train_7 = polynomial2.fit_transform(X_train_7)
X_test_7 = polynomial2.fit_transform(X_test_7)

scaler = StandardScaler()

# Scale everything except the constant term (column index 0)
X_train_7[:, 1:] = scaler.fit_transform(X_train_7[:, 1:])
X_test_7[:, 1:] = scaler.transform(X_test_7[:, 1:])

# Convert to DataFrame, maintaining the column names
cols = polynomial2.get_feature_names_out()
X_train_7 = pd.DataFrame(X_train_7, columns=cols, index=train_index)
X_test_7 = pd.DataFrame(X_test_7, columns=cols, index=test_index)

In [ ]:
model = sm.OLS(y_train, X_train_7)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_7)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_7.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \beta_3 \cdot ef \cdot creatinine + \beta_4 \cdot creatinine^2 + \epsilon $$

In [ ]:
X_train_8 = X_train_7.drop('ejection_fraction^2', axis=1)
X_test_8 = X_test_7.drop('ejection_fraction^2', axis=1)

In [ ]:
model = sm.OLS(y_train, X_train_8)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_8)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_8.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

## Categorical Features

To use them we need to transform categorical variables with one-hot encoding


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)

encoded_feature = encoder.fit_transform(df[cat_vars])
df_encoded = pd.DataFrame(encoded_feature, columns=encoder.get_feature_names_out(cat_vars))
df_encoded = pd.concat([df.drop(cat_vars, axis=1), df_encoded], axis=1)
df_encoded

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot death+ \epsilon $$

In [ ]:
num_variables = ['ejection_fraction']
cat_variables = ['DEATH_EVENT_True']

X_train_9_num = X_train[num_variables]
X_test_9_num = X_test[num_variables]

X_train_9_cat = df_encoded.loc[train_index, cat_variables]
X_test_9_cat = df_encoded.loc[test_index, cat_variables]


# Concatenate the polynomial features with the one-hot encoded categorical variable
X_train_9 = np.concatenate([X_train_9_num, X_train_9_cat], axis=1)
X_test_9 = np.concatenate([X_test_9_num, X_test_9_cat], axis=1)

cols = num_variables + cat_variables
X_train_9 = pd.DataFrame(X_train_9, index=train_index, columns=cols)
X_test_9 = pd.DataFrame(X_test_9, index=test_index, columns=cols)

X_train_9 = sm.add_constant(X_train_9)
X_test_9 = sm.add_constant(X_test_9)

In [ ]:
model = sm.OLS(y_train, X_train_9)
results = model.fit()
print(results.summary())

In [ ]:
X_plot = np.arange(np.min(df['ejection_fraction']), np.max(df['ejection_fraction']), 0.1).reshape(-1, 1)
X_plot = sm.add_constant(X_plot)

# Create two versions of X_plot, one with the binary variable = 0, one with binary variable = 1
X_plot_0 = np.hstack([X_plot, np.zeros((X_plot.shape[0], 1))])  # Add column of 0's for binary_var = 0
X_plot_1 = np.hstack([X_plot, np.ones((X_plot.shape[0], 1))])   # Add column of 1's for binary_var = 1

y_plot_0 = results.predict(X_plot_0)  # Prediction for binary_var = 0
y_plot_1 = results.predict(X_plot_1)  # Prediction for binary_var = 1

sns.relplot(x="ejection_fraction", y="bmi", height=6, data=df, legend=False, hue='DEATH_EVENT', palette={0: 'green', 1: 'red'}, alpha=.5)

sns.lineplot(x=X_plot[:, 1], y=y_plot_0, color='green', label='DEATH_EVENT_False')
sns.lineplot(x=X_plot[:, 1], y=y_plot_1, color='red', label='DEATH_EVENT_True')

In [ ]:
y_pred = results.predict(X_test_9)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_9.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \beta_3 \cdot ef \cdot creatinine + \beta_4 \cdot creatinine^2 + \beta_5 \cdot death+ \epsilon $$

In [ ]:
num_variables = ['ejection_fraction', 'serum_creatinine']
cat_variables = ['DEATH_EVENT_True']

X_train_9_num = X_train[num_variables]
X_test_9_num = X_test[num_variables]

X_train_9_cat = df_encoded.loc[train_index, cat_variables]
X_test_9_cat = df_encoded.loc[test_index, cat_variables]

# Perform Polynomial Features transformation on the numerical data
polynomial2 = PolynomialFeatures(degree=2, include_bias=True)
X_train_9_poly = polynomial2.fit_transform(X_train_9_num)
X_test_9_poly = polynomial2.fit_transform(X_test_9_num)

# Scale the polynomial features except for the bias (constant) term
scaler = StandardScaler()
X_train_9_poly[:, 1:] = scaler.fit_transform(X_train_9_poly[:, 1:])
X_test_9_poly[:, 1:] = scaler.transform(X_test_9_poly[:, 1:])

# Concatenate the polynomial features with the one-hot encoded categorical variable
X_train_9 = np.concatenate([X_train_9_poly, X_train_9_cat], axis=1)
X_test_9 = np.concatenate([X_test_9_poly, X_test_9_cat], axis=1)

# Convert to DataFrame for easier handling
cols_poly = polynomial2.get_feature_names_out(num_variables)
cols = np.concatenate([cols_poly, X_train_9_cat.columns])

X_train_9 = pd.DataFrame(X_train_9, columns=cols, index=train_index)
X_test_9 = pd.DataFrame(X_test_9, columns=cols, index=test_index)

X_train_9.drop('ejection_fraction^2', axis=1, inplace=True)
X_test_9.drop('ejection_fraction^2', axis=1, inplace=True)


In [ ]:
model = sm.OLS(y_train, X_train_9)
results = model.fit()
print(results.summary())

In [ ]:
y_pred = results.predict(X_test_9)
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
n = len(y_test)
p = X_test_9.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

# Diagnostics

Let's check the assumptions of linear regression.

Assumptions:

1. There is a linear relationship between the predictors and the response
2. $\epsilon_i$ (residuals) are indipendent
3. $Var(\epsilon_i) = \sigma^2 \quad \forall i $ (homoschedasticity)

and for inference:

4. $\epsilon_i \sim N(0,\sigma^2) \quad \forall i$ (normality) --- or at least large $n$ (sample size)

Let's consider the model

$$ bmi = \beta_0 + \beta_1 \cdot ef + \beta_2 \cdot creatinine + \beta_3 \cdot \text{creatinine_phosphokinase} + \epsilon $$

In [ ]:
variables = ['ejection_fraction', 'serum_creatinine', 'creatinine_phosphokinase']

X_train_5 = X_train[variables]
X_test_5 = X_test[variables]
X_train_5 = sm.add_constant(X_train_5)
X_test_5 = sm.add_constant(X_test_5)

In [ ]:
model = sm.OLS(y_train, X_train_5)
results = model.fit()
print(results.summary())

## Independence

Results summary includes Durbin-Watson test. The values of its test statistics are between 0 and 4. Values near 2 indicates no autocorrelation (independence), smaller values indicate positive autocorrelation, higher values negative autocorrelation

In [ ]:
from statsmodels.stats.stattools import durbin_watson

In [ ]:
durbin_watson(results.resid)

## Linearity and Homoschedasticity

We can visually inspect the residuals to check both

In [ ]:
fitted_vals = results.fittedvalues
residuals = results.resid

plt.figure(figsize=(8, 6))
sns.residplot(x=fitted_vals, y=residuals, lowess=True,
              line_kws={'color': 'red', 'lw': 1.5})  # Add a smooth trendline

plt.axhline(0, color='black', linestyle='--', lw=2)  # Horizontal line at 0
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values for Linearity & Homoscedasticity Check')
plt.show()

Breusch-Pagan statistical test for homoschedascity.

*H0: residuals are homoschedastic*

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

bp_test = het_breuschpagan(results.resid, X_train_5)

print(f'p-value: {bp_test[1]}')

## Gaussianity

Results summary includes the Jacques-Bera test for normality on residuals.

*H0: data are gaussian.*

This test has poor power for n < 100.

Other methods we previosly saw are the QQ-plot (graphical) and the Shapiro-Wilk test (good power for n < 2000)

In [ ]:
print(f'Jacques-Bera test p-value: {results.diagn["jbpv"]:.4f}')

In [ ]:
sw = stats.shapiro(results.resid).pvalue
print(f'Shapiro-Wilk test p-value: {sw:.4f}')

In [ ]:
stats.probplot(results.resid, dist="norm", plot=plt)
plt.show()

## Transform y

In [ ]:
y_train_10 = np.sqrt(y_train)
y_test_10 = np.sqrt(y_test)

# y_train_10 = np.log(y_train + 1)
# y_test_10 = np.log(y_test + 1)

# y_train_10 = np.cbrt(y_train)
# y_test_10 = np.cbrt(y_test)

model = sm.OLS(y_train_10, X_train_5)
results_new = model.fit()
print(results.summary())

In [ ]:
y_pred = results_new.predict(X_test_5)
r2 = r2_score(y_test_10,y_pred)
mse = mean_squared_error(y_test_10,y_pred)
n = len(y_test)
p = X_test_5.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'R2: {r2:.4f}')
print(f'adj_R2: {adj_r2:.4f}')
print(f'MSE: {mse:.4f}')

In [ ]:
sw = stats.shapiro(results_new.resid).pvalue
print(f'Shapiro-Wilk test p-value: {sw:.4f}')

In [ ]:
stats.probplot(results_new.resid, dist="norm", plot=plt)
plt.show()

## Leverages & outliers

**Outliers**: points with a very high residual (bad fit)
Residuals are standardized so to make it easier to spot outliers (those outside +- 2 or +- 3)

**Leverages**: points with unusual value

Hat Matrix $H$ s.t. $\hat{y}=Hy$

$H = X(X^TX)^{-1}X^T$

the leverage of point $i$ is $h_{ii} = x_i^T(X^TX)^{-1}x_i \in (0,1)$



**Cook's distanc**e: a mease of the point's overall influence of the model

$$
D_i = \frac{\epsilon_i^2}{k \cdot \sigma^2} \cdot \frac{h_{ii}}{(1-h_{ii})^2}
$$

comined the size of the (standardized) residuals, with respect to the model parameters, and the leverage

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sm.graphics.influence_plot(results, criterion="cooks", ax=ax)

In [ ]:
influence = results.get_influence()

influence_df = influence.summary_frame()[['cooks_d', 'student_resid', 'hat_diag']]

influence_df = influence_df.rename(columns={'hat_diag': 'leverage'}).sort_values(by='cooks_d', ascending=False)

influence_df

In [ ]:
X_train_5.loc[9]

## Removing high leverage outliers

In [ ]:
to_remove = [9]

X_train_11 = X_train_5.drop(to_remove)
y_train_11 = y_train.drop(to_remove)
X_test_11 = X_test_5
y_test_11 = y_test
model = sm.OLS(y_train_11, X_train_11)
results = model.fit()
print(results.summary())

In [ ]:
sw = stats.shapiro(results.resid).pvalue
print(f'Shapiro-Wilk test p-value: {sw:.4f}')

In [ ]:
stats.probplot(results.resid, dist="norm", plot=plt)
plt.show()

In [ ]:
fitted_vals = results.fittedvalues
residuals = results.resid

plt.figure(figsize=(8, 6))
sns.residplot(x=fitted_vals, y=residuals, lowess=True,
              line_kws={'color': 'red', 'lw': 1.5})  # Add a smooth trendline

plt.axhline(0, color='black', linestyle='--', lw=2)  # Horizontal line at 0
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values for Linearity & Homoscedasticity Check')
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

bp_test = het_breuschpagan(results.resid, X_train_11)

print(f'p-value: {bp_test[1]}')

# Anscombe dataset

Four datasets with nearly identical:

* Mean of
𝑥
x and
𝑦
y

* Variance of
𝑥
x and
𝑦
y

* Correlation between
𝑥
x and
𝑦
y

* Regression line (𝑦 = 3.00 + 0.50𝑥 )

* $𝑅^2$
  and residual standard error

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Load the built-in Anscombe dataset
df = sns.load_dataset("anscombe")

# Prepare containers
datasets = ['I', 'II', 'III', 'IV']
models = {}

# Plot setup: 2 rows x 4 columns
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Anscombe's Quartet: Regression & Residuals", fontsize=16)

for i, name in enumerate(datasets):
    data = df[df['dataset'] == name]

    x = data['x']
    y = data['y']
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    models[name] = model

    # Scatter plot with regression line (top row)
    ax1 = axes[0, i]
    ax1.scatter(x, y, color='blue')
    ax1.plot(x, model.predict(X), color='red')
    ax1.set_title(f'Dataset {i+1}')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')

    # Residual plot (bottom row)
    ax2 = axes[1, i]
    ax2.scatter(x, model.resid, color='#ff6600')
    ax2.axhline(0, color='gray', linestyle='--')
    ax2.set_title(f'Residuals {i+1}')
    ax2.set_xlabel('x')
    ax2.set_ylabel('Residuals')

# Show all plots
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Print regression summaries
for name, model in models.items():
    print(f"\n=== Summary for Dataset {name} ===")
    print(model.summary())


In [ ]:
## TODO: for datasets III and IV compute leverages and Cook distances
for dataset_label in ['III', 'IV']:
    print(f"Dataset {dataset_label}")

    # Filter data
    data = df[df['dataset'] == dataset_label]
    x = data['x']
    y = data['y']
    X = sm.add_constant(x)

    # Fit OLS model
    model = sm.OLS(y, X).fit()

    # Get influence measures
    influence = model.get_influence()
    leverage = influence.hat_matrix_diag
    cooks_d = influence.cooks_distance[0]  # [0] is Cook's D values, [1] are p-values

    # Create result DataFrame
    diagnostics_df = pd.DataFrame({
        'x': x,
        'y': y,
        'leverage': leverage,
        'cooks_distance': cooks_d,
        'residuals': model.resid,
        'standard_resid': influence.resid_studentized_internal
    }, index=data.index)

    print(diagnostics_df.round(3))

    # Influence plot
    fig, ax = plt.subplots(figsize=(6, 5))
    sm.graphics.influence_plot(model, ax=ax, criterion="cooks")
    plt.title(f'Influence Plot - Dataset {dataset_label}')
    plt.show()

# Bias-variance tradeoff

In [ ]:
from sklearn.metrics import r2_score

np.random.seed(1)

# True model
def f(x): return 1 + x + x**2 - x**3
sigma = 0.25

# Generate data
x = np.linspace(-1, 1.5, 21)
y = f(x) + np.random.normal(0, sigma, size=x.shape)
y_new = f(x) + np.random.normal(0, sigma, size=x.shape)

# Build design matrix up to x^20
X = np.column_stack([x**p for p in range(21)])
X_df = pd.DataFrame(X, columns=[f'x{p}' for p in range(21)])

# Grid for plotting
x_plot = np.linspace(-1, 1.5, 210)
X_plot = np.column_stack([x_plot**p for p in range(21)])
X_plot_df = pd.DataFrame(X_plot, columns=[f'x{p}' for p in range(21)])

# Plot data with true curve
plt.figure(figsize=(8, 5))
plt.plot(x, y, 'ko', label='Training Set')
plt.plot(x, y_new, 'ro', label='Test Set')
plt.plot(x_plot, f(x_plot), 'b--', label='True Mean')
plt.legend(loc='lower right')
plt.title("Training & Test Sets with True Function")
plt.grid(True)
plt.show()

# Metrics to store
SSres, SSres_new, s2, R2, R2_adj, coeffs_all = [], [], [], [], [], []
n = len(x)

# Fit and plot polynomial models
plt.figure(figsize=(16, 16))
for p in range(1, 17):
    X_train_p = X_df.iloc[:, :p+1]
    X_plot_p = X_plot_df.iloc[:, :p+1]

    model = sm.OLS(y, X_train_p).fit()
    y_pred = model.predict(X_train_p)
    y_pred_new = model.predict(X_train_p)
    y_plot = model.predict(X_plot_p)

    # Plot the fit
    plt.subplot(4, 4, p)
    plt.plot(x, y, 'ko', label='Train')
    plt.plot(x, y_new, 'ro', label='Test')
    plt.plot(x_plot, y_plot, 'k-', label='Model')
    plt.plot(x_plot, f(x_plot), 'b--', label='True')
    plt.title(f'{p}th Order Polynomial')
    plt.xticks([]); plt.yticks([])

    # Store metrics
    res = y - y_pred
    res_new = y_new - y_pred_new
    SSres.append(np.sum(res**2))
    SSres_new.append(np.sum(res_new**2))
    s2.append(np.sum(res**2) / (n - (p + 1)))
    r2 = r2_score(y, y_pred)
    R2.append(r2)
    R2_adj.append(1 - (1 - r2) * (n - 1) / (n - p - 1))

    # Coefficients
    coeffs = np.zeros(17)
    coeffs[:p+1] = model.params
    coeffs_all.append(coeffs)

plt.tight_layout()
plt.show()

# Coefficients matrix
b = np.array(coeffs_all).T

# Compare model performance
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(range(1, 17), SSres, 'ko-')
plt.axvline(3, color='blue', linestyle='--')
plt.title("Sum of Squared Residuals (Train)")

plt.subplot(2, 2, 2)
plt.plot(range(1, 17), SSres_new, 'ro-')
plt.axvline(3, color='blue', linestyle='--')
plt.title("Sum of Squared Residuals (Test)")

plt.subplot(2, 2, 3)
plt.plot(range(1, 17), R2, 'go-')
plt.axvline(3, color='blue', linestyle='--')
plt.title("R²")

plt.subplot(2, 2, 4)
plt.plot(range(1, 17), R2_adj, 'bo-')
plt.axvline(3, color='blue', linestyle='--')
plt.title("Adjusted R²")

plt.tight_layout()
plt.show()

# Compare coefficients
b_true = np.array([1, 1, 1, -1] + [0]*13)
plt.figure(figsize=(12, 8))
for i in range(5):
    plt.subplot(2, 3, i+1)
    plt.plot(range(1, 17), b[i], 'ko-')
    plt.axhline(b_true[i], color='blue')
    plt.axvline(3, color='blue', linestyle='--')
    plt.title(f'b[{i}]')
    plt.grid(True)
    if i == 0:
        plt.legend(['Estimated', 'True Coefficient', 'True Order'])

# Plot estimated variance
plt.subplot(2, 3, 6)
plt.plot(range(1, 17), s2, 'ko-')
plt.axhline(sigma**2, color='blue')
plt.axvline(3, color='blue', linestyle='--')
plt.title("Estimated σ²")
plt.grid(True)

plt.tight_layout()
plt.show()
